<a href="https://colab.research.google.com/github/drewcark/DS5500-drug-discovery/blob/main/Sreeja_2_12_25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/drewcark/DS5500-drug-discovery
%cd DS5500-drug-discovery

!git pull origin main

import zipfile
zip_fp = 'drugbank.csv.zip'
with zipfile.ZipFile(zip_fp, 'r') as zip_ref:
    zip_ref.extractall('/content/DS5500-drug-discovery/')

!unzip 'drugbank.csv.zip' - d /content/DS5500-drug-discovery/

zip_fp = 'smiles_updated.csv.zip'
with zipfile.ZipFile(zip_fp, 'r') as zip_ref:
    zip_ref.extractall('/content/DS5500-drug-discovery/')

!unzip 'smiles_updated.csv.zip' - d /content/DS5500-drug-discovery/

Cloning into 'DS5500-drug-discovery'...
remote: Enumerating objects: 90, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 90 (delta 31), reused 12 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (90/90), 80.95 MiB | 15.43 MiB/s, done.
Resolving deltas: 100% (31/31), done.
/content/DS5500-drug-discovery
From https://github.com/drewcark/DS5500-drug-discovery
 * branch            main       -> FETCH_HEAD
Already up to date.
Archive:  drugbank.csv.zip
caution: filename not matched:  -
caution: filename not matched:  d
caution: filename not matched:  /content/DS5500-drug-discovery/
Archive:  smiles_updated.csv.zip
caution: filename not matched:  -
caution: filename not matched:  d
caution: filename not matched:  /content/DS5500-drug-discovery/


In [2]:
import csv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy.optimize as opt
import seaborn as sns
from scipy import stats
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split
from tabulate import tabulate



ddi_fp = '/content/DS5500-drug-discovery/drugbank.csv'

ddi = pd.read_csv(ddi_fp, sep='\t')

kaggle_fp = '/content/DS5500-drug-discovery/smiles_updated.csv'

smiles = pd.read_csv(kaggle_fp, sep='\t')
smiles1 = smiles['SMILES,CHEMBL'].str.split(',', n=1, expand=True)
smiles1.columns = ['SMILES', 'CHEMBL']
smiles1.head()

#drug_names_fp = "drugs.txt"

#drug_names = pd.read_csv(drug_names_fp, sep='\t')

,SMILES,CHEMBL
0,C,CHEMBL17564
1,CO,CHEMBL14688
2,NCCS,CHEMBL602
3,NCCN,CHEMBL816
4,CN,CHEMBL43280


In [3]:
!pip install torch rdkit numpy scikit-learn

!pip install rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors, rdMolDescriptors
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect

import numpy as np
import pandas as pd


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.3/34.3 MB 16.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [4]:
def extract_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return None  # Handle invalid SMILES

    features = {
        "MolWt": Descriptors.MolWt(mol),  # Molecular weight
        "NumHDonors": Descriptors.NumHDonors(mol),  # Hydrogen bond donors
        "NumHAcceptors": Descriptors.NumHAcceptors(mol),  # Hydrogen bond acceptors
        "NumRotatableBonds": Descriptors.NumRotatableBonds(mol),  # Flexibility
        "TPSA": Descriptors.TPSA(mol),  # Topological polar surface area
        "MorganFP": list(GetMorganFingerprintAsBitVect(mol, 2, nBits=2048))  # Morgan FP (radius=2)
    }
    return features


In [ ]:
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning, message="please use MorganGenerator")



Streaming output truncated to the last 5000 lines.
[16:44:19] DEPRECATION WARNING: please use MorganGenerator
[16:44:19] DEPRECATION WARNING: please use MorganGenerator
[16:44:19] DEPRECATION WARNING: please use MorganGenerator
[16:44:19] DEPRECATION WARNING: please use MorganGenerator
[16:44:19] DEPRECATION WARNING: please use MorganGenerator
[16:44:19] DEPRECATION WARNING: please use MorganGenerator
[16:44:19] DEPRECATION WARNING: please use MorganGenerator
[16:44:19] DEPRECATION WARNING: please use MorganGenerator
[16:44:19] DEPRECATION WARNING: please use MorganGenerator
[16:44:19] DEPRECATION WARNING: please use MorganGenerator
[16:44:19] DEPRECATION WARNING: please use MorganGenerator
[16:44:19] DEPRECATION WARNING: please use MorganGenerator
[16:44:19] DEPRECATION WARNING: please use MorganGenerator
[16:44:19] DEPRECATION WARNING: please use MorganGenerator
[16:44:19] DEPRECATION WARNING: please use MorganGenerator
[16:44:19] DEPRECATION WARNING: please use MorganGenerator
[16:4